In [3]:
%config InlineBackend.figure_format = 'retina'
import numpy as np
import pandas as pd
import circ_corr as cc
import seaborn as sns
import matplotlib.pyplot as plt
from multiprocessing import Pool
import scipy
pi=np.pi
subjs = ['UCSD0' + x for x in ['54','60','61','62','63','64']]
rois = ['V1','V2','V3','V3AB','hV4','IPS0']
n_subj=len(subjs)

def sem_plot(x,y,axs=0,within_E=0,do_line=1,**args):
    # x is assumed to be examples x len(y)
    n_ex,n_points =  y.shape
    if n_points!=len(x):
        y=y.T
        n_ex,n_points =  y.shape
    assert n_points==len(x), 'x not correct shape'
    m_y = np.mean(y,0)
    if within_E:
        y_use = (y.T-np.mean(y,1)).T
        s_y = np.std(y_use,0)/np.sqrt(n_ex)
    else:
        s_y = np.std(y,0)/np.sqrt(n_ex)
    if axs==0:
        plt.fill_between(x,m_y-s_y,m_y+s_y,**args)
        if do_line:
            plt.plot(x,m_y,'k')
    else:
        axs.fill_between(x,m_y-s_y,m_y+s_y,**args)
        if do_line:
            axs.plot(x,m_y,'k')
    
import matplotlib 
font = {'family' : 'DejaVu Sans',
        'weight' : 'normal',
        'size'   : 16}

matplotlib.rc('font', **font)

import matplotlib
cmap = matplotlib.cm.get_cmap('Dark2')
ct = cmap(np.linspace(0,1,4))

In [2]:
def sav_fig(nam):
    root = './Figs_v5/'
    plt.savefig(root + nam +'.svg',dpi=1200)
    
def circ_mean_rad(x): return np.angle(np.mean(np.exp(1j*x))) # rad --> rad
def circ_mean(x): return np.angle(np.mean(np.exp(1j*x/90*pi)))*90/pi # deg --> deg
def circ_std(x):
    R = np.abs(np.mean(np.exp(1j*x/90*pi)))
    v = np.sqrt(-2*np.log(R))
    return v
def inv_cdf(cdf,probe): return np.argmin(np.abs(np.expand_dims(cdf,1) - probe),0) # draws from 'y'-axis

# To Implement:
1. fitting a subject iteratively with cross-validation
1. evaluating goodness of fit(?) relative to null model?

Later Add
1. Fitting function 1 using max likelihood! 
1. Poisson noise



# Helper Functions

In [4]:
def do_bining(bns,overlap,grouping_var,var,var2=[-999],flip=1,prior_gauss=[4,0],want_var=0):
#     global obs, d_use_rad
    n_bns=len(bns)
    grouper = np.zeros(len(bns))
    if var2[0]!=-999:
        do_SDT=1

        out = np.zeros((2,len(bns)))
            
        if flip:
            grouping_var = np.concatenate((grouping_var,-grouping_var))
            var = np.concatenate((var,(var==0)*1))
            var2 = np.concatenate((var2,-var2))
    else:
        do_SDT=0
        out = np.zeros(len(bns))
        if flip:
            grouping_var = np.concatenate((grouping_var,-grouping_var))
            if flip==2:
                var = np.concatenate((var,var))
            elif flip==1:
                var = np.concatenate((var,-var))
    for i in range(n_bns):
        if overlap==0:
            if i==(n_bns-1):
                continue
            else:
                these_ind = (grouping_var>=bns[i])&(grouping_var<bns[i+1])
            
        elif i<overlap:
            these_ind = (grouping_var<=bns[i+overlap]) | (grouping_var>=bns[i-overlap])
        elif i>(n_bns-overlap-1):
            these_ind = (grouping_var>=bns[i-overlap]) | (grouping_var<=bns[i+overlap-n_bns])
        else:
            these_ind = (grouping_var>=bns[i-overlap])&(grouping_var<=bns[i+overlap]) # need to figure out 
            
        if do_SDT:
            dat_in = (var[these_ind],var2[these_ind])
            fit = scipy.optimize.minimize(min_fun_gauss,prior_gauss,args=(dat_in,),bounds=((0.01,45),(-45,45)) )
            out[:,i] = [fit.x[0],-fit.x[1]]
 
        else:
            if want_var==0:
                out[i] = np.mean(var[these_ind])
            elif want_var==1:
                out[i] = np.std(var[these_ind])
            elif want_var==2:
                out[i] = circ_std(var[these_ind])
            elif want_var==3:
                out[i] = circ_mean(var[these_ind])
            elif want_var==4:
                out[i] = np.sum(wrap(var[these_ind]-circ_mean(var[these_ind]))**2)
            
    return out

def wrap(x):
    x[np.abs(x)>90]-=180*np.sign(x[np.abs(x)>90])
    return x

def wrap_rad(x):
    x[np.abs(x)>pi]-=2*pi*np.sign(x[np.abs(x)>pi])
    return x

GUESS_RATE = 0.25
# def min_fun_gauss(p):
#     G = scipy.stats.norm(p[1],p[0])
#     lik = ((1-obs)*G.cdf(d_use_rad)+(obs)*(1-G.cdf(d_use_rad)))*(1-GUESS_RATE)+0.5*GUESS_RATE
#     return -np.sum(np.log(lik))

def min_fun_gauss(p,dat_in):
    # sd, bias
    obs,d_use_deg = dat_in
    G = scipy.stats.norm(p[1],p[0])
    lik = ((1-obs)*G.cdf(d_use_deg)+(obs)*(1-G.cdf(d_use_deg)))*(1-GUESS_RATE)+0.5*GUESS_RATE
    return -np.sum(np.log(lik))

def gauss(x,mu=0,sig=1): return 1/np.sqrt(2*pi*sig**2)*np.exp(-((x-mu)**2)/(2*sig**2))
def std_e(x): return np.std(x,0)/np.sqrt(len(x))
def rectify(x):
    if do_rectify:
        return np.minimum(0,x)
    else:
        return x


# Model Definitions

In [5]:
# Model defs
def poisson_expect(expect,actual): return expect**actual/scipy.special.factorial(actual) * np.exp(-expect)
def c(s_0,s_1=0,lam=10/90*pi,gamma=2): ## eq 7
    full = np.exp(-1/(2*lam**2)*np.abs(wrap_rad(s_0-s_1))**gamma)
    return full/np.sum(full,0)*len(s_0)

def cf(s_0,s_1=0,lam=20/90*pi,gamma=2,p_same=0.64): ## eq 6  #10, 2, 0.9 (note p_same from BB&J2019 empiricial prior)
    return p_same*c(s_0,s_1,lam,gamma) + (1-p_same)

def run_model_1(p,data,min_mode='RSS',do_poisson=0):
    # p - params 
    # data - dictionary
    # min_mode 
    # do_poisson - flag, value is Rate
    
    gam_1, gam_s = p # unpack
    this_ori_prev,this_ori_current,this_ori_dec = data['ori_prev'],data['ori_current'],data['ori_dec']

    # implement adaptation on population
#     adapt_shape = np.minimum(0,-gam_1*np.cos(wrap_rad(Phi-this_ori_prev)*gam_s)**3) 
    adapt_shape = rectify(-gam_1*np.cos(wrap_rad(Phi-this_ori_prev)*gam_s)**3)     #- adaptation shape (neuron x trial)
    gamma_a =adapt_shape+gamma_n -np.mean(adapt_shape,0)*add_constant_adapt #- normalized gain (neuron x trial)
    
    resp_a_stim = gamma_a*np.exp(kappa_a*np.cos(wrap_rad(Phi-this_ori_current))-1)  #- adapted response (neuron x trial)
    
    if do_poisson:
        ll_n_a_all = np.sum(np.log(poisson_expect(TC_n*do_poisson,np.expand_dims(resp_a_stim*do_poisson,2))),0)
    else:
        ll_n_a_all = np.sum(np.log(gauss(TC_n-np.expand_dims(resp_a_stim,2),0,sd)),0) # LL @ all ori (trial x orientation)

    if min_mode=='RSS': #     for minimizing L2 error
        max_lik = ori[np.argmax(ll_n_a_all,1)]
        e_model = wrap_rad(max_lik-this_ori_dec)
        RSS = np.mean(e_model**2)
        return RSS
    
    if min_mode=='decode':
        max_lik = ori[np.argmax(ll_n_a_all,1)]
        return max_lik
    
    if min_mode=='vis':
        max_lik = ori[np.argmax(ll_n_a_all,1)]
        E_this = wrap_rad(max_lik-this_ori_current)*90/pi
        return E_this

def run_model_2(p,data,min_mode='maxlik_gauss',model_mode='a_a',do_poisson=0):
    
    this_ori_prev,this_ori_current,this_ori_dec = data['ori_prev'],data['ori_current'],data['ori_dec']
    this_probe,this_resp = data['probe'], data['resp']
    gam_1,gam_s = data['fit_1']
    if model_mode!='a_a_2':
        if do_poisson:
            do_poisson,lam = p
            sd = np.nan
        else:
            sd, lam = p
        prior = cf(np.expand_dims(ori,1),s_1=this_ori_prev,lam=lam).T              # (trial x orientation)
    elif model_mode=='a_a_2': # double aware
        if do_poisson:
            do_poisson=5
        else:
            sd = 0.15
        gam_1_2, gam_s_2 = p
        # adapt_shape_2 = np.minimum(0,-gam_1_2*np.cos(wrap_rad(Phi-this_ori_prev)*gam_s_2)**3)    
        adapt_shape_2 = rectify(-gam_1_2*np.cos(wrap_rad(Phi-this_ori_prev)*gam_s_2)**3)       #- adaptation shape (neuron x trial)
        gamma_a_2 =adapt_shape_2+gamma_n -np.expand_dims(np.mean(adapt_shape_2,0),0)*add_constant_adapt #- normalized gain (neuron x trial)
        prior = 1 #- multiplicative identity
        
#     adapt_shape = np.minimum(0,-gam_1*np.cos(wrap_rad(Phi-this_ori_prev)*gam_s)**3)    
    adapt_shape = rectify(-gam_1*np.cos(wrap_rad(Phi-this_ori_prev)*gam_s)**3)       #- adaptation shape (neuron x trial)
    gamma_a =adapt_shape+gamma_n -np.expand_dims(np.mean(adapt_shape,0),0)*add_constant_adapt #- normalized gain (neuron x trial)
    resp_a_stim = gamma_a*np.exp(kappa_a*np.cos(wrap_rad(Phi-this_ori_current))-1)  #- adapted response (neuron x trial)
    
    if model_mode=='a_a':
        TC_a = np.expand_dims(gamma_a,2)*np.expand_dims(np.exp(kappa_a*np.cos(Phi-ori)-1),1) # expected response adapt ()
        if do_poisson:
            ll_a_a = np.sum(np.log(poisson_expect(TC_a*do_poisson,np.expand_dims(resp_a_stim*do_poisson,2))),0)
        else:
            ll_a_a = np.sum(np.log(gauss(TC_a-np.expand_dims(resp_a_stim,2),0,sd)),0)  # (trial x orientation)
        ll_use = ll_a_a
    elif model_mode=='a_n':
        if do_poisson:
            ll_a_n = np.sum(np.log(poisson_expect(TC_n*do_poisson,np.expand_dims(resp_a_stim*do_poisson,2))),0)
        else:
            ll_a_n = np.sum(np.log(gauss(TC_n-np.expand_dims(resp_a_stim,2),0,sd)),0)  # (trial x orientation)
        ll_use = ll_a_n
    elif model_mode=='a_a_2': # double aware
        TC_a_2 = np.expand_dims(gamma_a_2,2)*np.expand_dims(np.exp(kappa_a*np.cos(Phi-ori)-1),1) # expected response adapt ()
        if do_poisson:
            ll_a_a_2 = np.sum(np.log(poisson_expect(TC_a_2*do_poisson,np.expand_dims(resp_a_stim*do_poisson,2))),0)
        else:
            ll_a_a_2 = np.sum(np.log(gauss(TC_a_2-np.expand_dims(resp_a_stim,2),0,sd)),0)  # (trial x orientation)
        ll_use = ll_a_a_2
    else: raise
        
    ll_use_cap = ll_use-np.expand_dims(np.max(ll_use,1),1)                     # prevent numerical underflow (set max lik==1)
    posterior = np.exp(ll_use_cap)*prior
    max_post = ori[np.argmax(posterior,1)] # MLE (trial)
    
    if min_mode=='maxlik_gauss': #   takes max likelihood and applies gaussian kernel
        d_probe_post = wrap_rad(max_post-this_probe)*90/pi
        G = scipy.stats.norm(0,fit_sd) # fit_sd fit_sd_eRecon
        p_cw = G.cdf(d_probe_post)
        lik = (p_cw*(this_resp==0)+(this_resp==1)*(1-p_cw))*(1-GUESS_RATE) + 0.5*GUESS_RATE
        lik_resp  = np.sum(np.log(lik))
        return -lik_resp
    if min_mode == 'decode':
        d_probe_post = wrap_rad(max_post-this_probe)*90/pi
        G = scipy.stats.norm(0,fit_sd) # fit_sd fit_sd_eRecon
        p_cw = G.cdf(d_probe_post)
        lik = (p_cw*(this_resp==0)+(this_resp==1)*(1-p_cw))*(1-GUESS_RATE) + 0.5*GUESS_RATE
        return lik
    if min_mode == 'vis':
        E_this = wrap_rad(max_post-this_ori_current)*90/pi
        return E_this

In [6]:
def run_model_sim_all(p,data,get_act=0):
    # assume poisson
    gam_1, gam_s = p[0]
    do_poisson0,lam0 = p[1] # a_a
    do_poisson1,lam1 = p[2] # a_n
    gam_1_2, gam_s_2 = p[3] # a_a_2
    this_ori_prev,this_ori_current,this_ori_dec = data['ori_prev'],data['ori_current'],data['ori_dec']
    poisson_def = 5
    
    # encoding all
    # adapt_shape = np.minimum(0,-gam_1*np.cos(wrap_rad(Phi-this_ori_prev)*gam_s)**3)
    adapt_shape = rectify(-gam_1*np.cos(wrap_rad(Phi-this_ori_prev)*gam_s)**3)     #- adaptation shape (neuron x trial)
    gamma_a =adapt_shape+gamma_n -np.expand_dims(np.mean(adapt_shape,0),0)*add_constant_adapt #- normalized gain (neuron x trial)
    resp_a_stim = gamma_a*np.exp(kappa_a*np.cos(wrap_rad(Phi-this_ori_current))-1)
    
    if get_act:
        resp_n_stim = gamma_n*np.exp(kappa_n*np.cos(wrap_rad(Phi-this_ori_current))-1)
        return (np.sum(np.random.poisson(resp_n_stim*do_poisson),0),np.sum(np.random.poisson(resp_a_stim*do_poisson),0))
    
       
    # encoding readout
    resp_a_sim_enc =np.random.poisson(resp_a_stim*poisson_def)
    ll_n_a_enc = np.sum(np.log(poisson_expect(TC_n*poisson_def, np.expand_dims(resp_a_sim_enc,2))),0) # LL @ all ori (trial x orientation)
    TC_a = np.expand_dims(gamma_a,2)*np.expand_dims(np.exp(kappa_a*np.cos(Phi-ori)-1),1) # expected response adapt ()
    
    # prior
    prior0 = cf(np.expand_dims(ori,1),s_1=this_ori_prev,lam=lam0).T              # (trial x orientation)
    prior1 = cf(np.expand_dims(ori,1),s_1=this_ori_prev,lam=lam1).T              # (trial x orientation)
    
    # a_a
    resp_a_sim_a_a =np.random.poisson(resp_a_stim*do_poisson0)
    ll_a_a_dec = np.sum(np.log(poisson_expect(TC_a*do_poisson0,np.expand_dims(resp_a_sim_a_a,2))),0)
    
    # a_n
    resp_a_sim_a_n =np.random.poisson(resp_a_stim*do_poisson1)
    ll_n_a_dec = np.sum(np.log(poisson_expect(TC_n*do_poisson1, np.expand_dims(resp_a_sim_a_n,2))),0) # LL @ all ori (trial x orientation)
    
    # a_a_2
#     adapt_shape_2 = np.minimum(0,-gam_1_2*np.cos(wrap_rad(Phi-this_ori_prev)*gam_s_2)**3)   
    adapt_shape_2 = rectify(-gam_1_2*np.cos(wrap_rad(Phi-this_ori_prev)*gam_s_2)**3)       #- adaptation shape (neuron x trial)
    gamma_a_2 =adapt_shape_2+gamma_n -np.expand_dims(np.mean(adapt_shape_2,0),0)*add_constant_adapt #- normalized gain (neuron x trial)
    TC_a_2 = np.expand_dims(gamma_a_2,2)*np.expand_dims(np.exp(kappa_a*np.cos(Phi-ori)-1),1) # expected response adapt ()
    ll_a_a_2_dec = np.sum(np.log(poisson_expect(TC_a_2*poisson_def,np.expand_dims(resp_a_sim_enc,2))),0)
    
    ll_all = [ll_n_a_enc,ll_a_a_dec,ll_n_a_dec,ll_a_a_2_dec]
    ll_all_cap = [l - np.expand_dims(np.max(l,1),1) for l in ll_all]
    prior_all = [1,prior0,prior1,1]
    post_all = [np.exp(ll_all_cap[i])*prior_all[i] for i in range(4)]
    max_ll_all = [ori[np.argmax(post,1)] for post in post_all]
    E_all = [wrap_rad(this_maxLL-this_ori_current)*90/pi for this_maxLL in max_ll_all]
    return E_all


# Initialize Model Parameters

In [7]:
N_units=100
N_stim=91
N_probe=720 # 'resolution of model', 360 is good, go higher (1440) for publication quality figs

kappa_n = 1.0 # (0.25) # note this is wider than normal units but fine here for simplicity
gamma_n = 1.0
kappa_1 = 0.0
kappa_a = kappa_n
                              
Phi = np.expand_dims(np.linspace(-pi,pi-(2*pi/N_units),N_units),1) #- tuning curve centers
theta_n = np.linspace(-pi,pi-(pi/N_stim),N_stim)[::-1]           #- stimulus probes
ori = np.linspace(-pi,pi-(pi/N_probe),N_probe)                   #- stimuli to sample for likelihood

# for model visualization
theta_n1 = 0 # adapting stim (rad) 
resp_n = gamma_n*np.exp(kappa_n*np.cos(wrap_rad(Phi-theta_n))-1)  #- non-adapted response (for vis model)
TC_n = np.expand_dims(gamma_n*np.exp(kappa_n*np.cos(wrap_rad(Phi-ori))-1),1) # expected non-adapted response (for vis model)

default_var = {'gam_1':0.15,'gam_s':0.6,'sd':0.15,'lam':0.6}
# ranges_brute = (((0.00,0.9),(0.2,1.0)),  # P1
#                 ((0.05,0.50),(0.1,0.9)), # P2
#                 ((0.05,0.50),(0.1,0.9)), # P2 unaware
#                 ((0.00,0.90),(0.2,1.0))) # P2 overaware

# ranges_brute_poisson = (((0.1,0.9),(0.2,1.5)),  # P1
#                 ((5,45),(0.1,1.0)), # P2
#                 ((5,45),(0.1,1.0)), # P2 unaware
#                 ((0.1,0.90),(0.2,1.5))) # P2 overaware

ranges_brute_poisson = (((0.1,0.9),(0.1,1.5)),  # P1
                ((.5,10),(0.1,1.1)), # P2
                ((.01,10),(0.1,1.5)), # P2 unaware
                ((0.1,0.90),(0.1,1.5))) # P2 overaware

def_vals_poisson = ((0.3,1.0),(10.0,0.5),(0.5,1.0),(0.5,1.0))
N_brute = 8
sd = 0.15 # default
add_constant_adapt = 0
do_rectify = 1

# Define CV Process

In [8]:
# make key parts into a function that can be parallelized
do_poisson = 5
if do_poisson:
    ranges_brute_use = ranges_brute_poisson
else:
    ranges_brute_use = ranges_brute
def do_CV(g):

    print(g, 'Starting')
    these_ind = G!=g
    data_in = {'ori_prev':ori_prev[:,these_ind],'ori_current':ori_current[these_ind],'ori_dec':ori_dec[these_ind],
               'probe':probe[these_ind],'resp':resp[these_ind]}
    
    # Brute --> NM
    out_brute_1 = scipy.optimize.brute(run_model_1,ranges_brute_use[0],Ns=N_brute,args=(data_in,'RSS',do_poisson),finish=None)
    out_this_1 = scipy.optimize.minimize(run_model_1,out_brute_1,args=(data_in,'RSS',do_poisson),method='Nelder-mead')
    print(g, 'part 1 complete')

    data_in['fit_1'] = out_this_1.x
    out_brute_2 = scipy.optimize.brute(run_model_2,ranges_brute_use[1],Ns=N_brute,args=(data_in,'maxlik_gauss','a_a',do_poisson),finish=None)
    out_this_2 = scipy.optimize.minimize(run_model_2,out_brute_2,args=(data_in,'maxlik_gauss','a_a',do_poisson),method='Nelder-mead')
    
    print(g, 'part 2 standard complete')
    out_brute_2_ua = scipy.optimize.brute(run_model_2,ranges_brute_use[2],Ns=N_brute,args=(data_in,'maxlik_gauss','a_n',do_poisson),finish=None)
    out_this_2_ua = scipy.optimize.minimize(run_model_2,out_brute_2_ua,args=(data_in,'maxlik_gauss','a_n',do_poisson),method='Nelder-mead')
    
    print(g, 'part 2 unaware complete')
    out_brute_2_oa = scipy.optimize.brute(run_model_2,ranges_brute_use[3],Ns=N_brute,args=(data_in,'maxlik_gauss','a_a_2',do_poisson),finish=None)
    out_this_2_oa = scipy.optimize.minimize(run_model_2,out_brute_2_oa,args=(data_in,'maxlik_gauss','a_a_2',do_poisson),method='Nelder-mead')
    
    print(g, 'part 2 overaware complete')
    return (out_this_1,out_this_2,out_this_2_ua,out_this_2_oa)
#     return (out_this_1.x,out_this_2.x,out_this_2_ua.x,out_this_2_oa.x)

# Fit with finer sampling over brute!

# Run each CV/ model efficiently/ in parallel!

In [9]:
# run part 1
def job_runner_1(g):
    print(g)
    these_ind = G!=g
    data_in = {'ori_prev':ori_prev[:,these_ind],'ori_current':ori_current[these_ind],'ori_dec':ori_dec[these_ind],
           'probe':probe[these_ind],'resp':resp[these_ind]}
    this_start = all_brute_runs[int(g),0,:]
    this_fit = scipy.optimize.minimize(run_model_1,this_start,args=(data_in,'RSS',do_poisson),method='Nelder-mead')
    print(g,'Done')
    return this_fit

# run parts 2-4
def job_runner(ind): # all decoding laps
    g = ind%n_g
    this_job = ind//n_g
    these_params = all_brute_runs[int(g),this_job+1,:]

    these_ind = G!=g
    data_in = {'ori_prev':ori_prev[:,these_ind],'ori_current':ori_current[these_ind],'ori_dec':ori_dec[these_ind],
           'probe':probe[these_ind],'resp':resp[these_ind]}
    data_in['fit_1'] = part_1_params[g]
    
    if this_job==0: # B-A
        out_this_2 = scipy.optimize.minimize(run_model_2,these_params,args=(data_in,'maxlik_gauss','a_a',do_poisson),method='Nelder-mead')
    elif this_job==1: # B-uA
        out_this_2 = scipy.optimize.minimize(run_model_2,these_params,args=(data_in,'maxlik_gauss','a_n',do_poisson),method='Nelder-mead')
    elif this_job==2:
        out_this_2 = scipy.optimize.minimize(run_model_2,these_params,args=(data_in,'maxlik_gauss','a_a_2',do_poisson),method='Nelder-mead')
    print(g,this_job,out_this_2.x)
    return out_this_2

n_g = len(np.unique(G))
n_job = n_g*3
nPool = 30
N_brute = 30

# run brute fit for all CV/models
all_brute_runs = []
for g in np.unique(G):
    print(g)
    these_ind = G!=g
    data_in = {'ori_prev':ori_prev[:,these_ind],'ori_current':ori_current[these_ind],'ori_dec':ori_dec[these_ind],
               'probe':probe[these_ind],'resp':resp[these_ind]}

    out_brute_1 = scipy.optimize.brute(run_model_1,ranges_brute_use[0],Ns=N_brute,args=(data_in,'RSS',do_poisson),finish=None,workers=nPool)
   
    print(g, 'part 1 complete')
    data_in['fit_1'] = out_this_1.x
    out_brute_2 = scipy.optimize.brute(run_model_2,ranges_brute_use[1],Ns=N_brute,args=(data_in,'maxlik_gauss','a_a',do_poisson),finish=None,workers=nPool)
   
    print(g, 'part 2 aware complete')
    out_brute_2_ua = scipy.optimize.brute(run_model_2,ranges_brute_use[2],Ns=N_brute,args=(data_in,'maxlik_gauss','a_n',do_poisson),finish=None,workers=nPool)
  
    print(g, 'part 2 unaware complete')
    out_brute_2_oa = scipy.optimize.brute(run_model_2,ranges_brute_use[3],Ns=N_brute,args=(data_in,'maxlik_gauss','a_a_2',do_poisson),finish=None,workers=nPool)
    
    print(g, 'part 2 overaware complete')

    all_brute_runs.append((out_brute_1,out_brute_2,out_brute_2_ua,out_brute_2_oa))
all_brute_runs = np.array(all_brute_runs)

# run fine tune fit for P1
with Pool(n_g) as pool:
    fit_part_1 = pool.map(job_runner_1,np.unique(G))
part_1_params = np.array([f.x for f in fit_part_1])

# run fine tune fit for P2-4
with Pool(20) as pool:
    fit_part_2 = pool.map(job_runner,range(n_job))
    
final_fits_part_2 = np.array([f.x for f in fit_part_2]).reshape(3,n_g,2)
params = np.swapaxes(np.concatenate((np.expand_dims(part_1_params,0),final_fits_part_2)),0,1)

NameError: name 'G' is not defined

# Load All Data

In [11]:
dat = pd.read_pickle('DAT_IEM_COMPRESS')

d_ori = np.concatenate(([0],wrap(dat.orient0[:-1].values - dat.orient0[1:].values)))
d_ori[dat.trial==0] = np.nan
dat['d_ori']=d_ori
d_stim = wrap(dat.orient0.values-dat.orient1.values)
dat['d_stim'] = d_stim

# Select subject/ ROI

In [13]:
rois[2:3]

['V3']

In [21]:
n_bns = 91
overlap = 4 
bns = np.linspace(-90,90,n_bns)

DEC_ALL = pd.Series()
MF = pd.DataFrame()
do_poisson = 5


for subj in subjs:
    dec_subj = pd.Series()
    for roi in rois[2:3]:#rois[2:3]:
        print(subj)

        SA = dat[(dat.subj==subj)]#&(dat.trial!=1)]
        good = (SA.trial!=1)&(~np.isnan(SA.respCW))

        # get CV group
        sess_u=SA.sess.unique()
        n_sess = len(sess_u)
        G_task=np.ones(len(SA))
        for i,gi in enumerate(np.arange(0,n_sess,4)):
            G_task[np.isin(SA.sess,sess_u[gi:gi+4])] = i
        G = G_task[good] # CV Groups!
        n_g = len(np.unique(G))

        ori_current = (SA.orient0.values/90*pi)[good]
        ori_prev = np.expand_dims(np.concatenate(([0],SA.orient0[:-1].values))/90*pi,0)[:,good]
        ori_dec = (SA['estIEM'+roi].values/90*pi-pi)[good]
        #         d_ori = wrap_rad(ori_prev.flatten()-ori_current)
        d_ori = SA['d_ori'].values[good]
        probe = SA.orient1.values[good]/90*pi
        resp = SA.respCW.values[good]
        eRecon = wrap(SA['estIEM'+roi].values-SA['orient0'].values-90)[good]
        obs = SA.respCW.values[good]
        d_use_deg = SA['d_stim'].values[good] # need to be in deg for gaussian

        obs_flip = obs.copy()*1
        obs_flip[d_ori<0] = 1-obs_flip[d_ori<0]
        d_use_deg_flip = d_use_deg.copy()
        d_use_deg_flip[d_ori<0] = d_use_deg_flip[d_ori<0]*-1

        out_percept = do_bining(bns,overlap,d_ori,resp,d_use_deg,prior_gauss=(3,0))
        out_neural = do_bining(bns,overlap,d_ori,eRecon)

        p_gauss = (4,0)
        dat_in = (obs_flip,d_use_deg_flip)
        fit_gauss = scipy.optimize.minimize(min_fun_gauss,p_gauss,args=(dat_in,),method='Nelder-Mead')
        fit_sd,fit_bias = fit_gauss.x
        ## - here putting in code (don't want to be in function)

        ### my block
        n_g = len(np.unique(G))
        n_job = n_g*3
        nPool = 30
        N_brute = 10

        # run brute fit for all CV/models
        all_brute_runs = []
        for g in np.unique(G):
            print(g)
            these_ind = G!=g
            data_in = {'ori_prev':ori_prev[:,these_ind],'ori_current':ori_current[these_ind],'ori_dec':ori_dec[these_ind],
                       'probe':probe[these_ind],'resp':resp[these_ind]}

            out_brute_1 = scipy.optimize.brute(run_model_1,ranges_brute_use[0],Ns=N_brute,args=(data_in,'RSS',do_poisson),finish=None,workers=nPool)

            print(g, 'part 1 complete')
            data_in['fit_1'] = out_brute_1
            out_brute_2 = scipy.optimize.brute(run_model_2,ranges_brute_use[1],Ns=N_brute,args=(data_in,'maxlik_gauss','a_a',do_poisson),finish=None,workers=nPool)

            print(g, 'part 2 aware complete')
            out_brute_2_ua = scipy.optimize.brute(run_model_2,ranges_brute_use[2],Ns=N_brute,args=(data_in,'maxlik_gauss','a_n',do_poisson),finish=None,workers=nPool)

            print(g, 'part 2 unaware complete')
            out_brute_2_oa = scipy.optimize.brute(run_model_2,ranges_brute_use[3],Ns=N_brute,args=(data_in,'maxlik_gauss','a_a_2',do_poisson),finish=None,workers=nPool)

            print(g, 'part 2 overaware complete')

            all_brute_runs.append((out_brute_1,out_brute_2,out_brute_2_ua,out_brute_2_oa))
        all_brute_runs = np.array(all_brute_runs)

        # run fine tune fit for P1
        with Pool(n_g) as pool:
            fit_part_1 = pool.map(job_runner_1,np.unique(G))
        part_1_params = np.array([f.x for f in fit_part_1])

        # run fine tune fit for P2-4
        with Pool(20) as pool:
            fit_part_2 = pool.map(job_runner,range(n_job))

        final_fits_part_2 = np.array([f.x for f in fit_part_2]).reshape(3,n_g,2)
        params = np.swapaxes(np.concatenate((np.expand_dims(part_1_params,0),final_fits_part_2)),0,1)
        ### my block

        data = {'ori_prev':ori_prev,'ori_current':ori_current,'ori_dec':ori_dec,
               'probe':probe,'resp':resp}

        dec_ori,lik_resp,dec_resp = np.zeros(len(ori_current)),np.zeros((3,len(ori_current))),np.zeros((3,len(ori_current)))
        for g in range(n_g):
            these_ind = G==g
            data_in = {'ori_prev':ori_prev[:,these_ind],'ori_current':ori_current[these_ind],'ori_dec':ori_dec[these_ind],
                       'probe':probe[these_ind],'resp':resp[these_ind]}

            data_in['fit_1'] = params[g,0,:]

            dec_ori[these_ind] = run_model_1(params[g,0,:],data_in,min_mode='decode')
            lik_resp[0,these_ind] = run_model_2(params[g,1,:],data_in,min_mode='decode',model_mode='a_a',do_poisson=do_poisson)
            lik_resp[1,these_ind] = run_model_2(params[g,2,:],data_in,min_mode='decode',model_mode='a_n',do_poisson=do_poisson)
            lik_resp[2,these_ind] = run_model_2(params[g,3,:],data_in,min_mode='decode',model_mode='a_a_2',do_poisson=do_poisson)

            dec_resp[0,these_ind] = run_model_2(params[g,1,:],data_in,min_mode='vis',model_mode='a_a',do_poisson=do_poisson)
            dec_resp[1,these_ind] = run_model_2(params[g,2,:],data_in,min_mode='vis',model_mode='a_n',do_poisson=do_poisson)
            dec_resp[2,these_ind] = run_model_2(params[g,3,:],data_in,min_mode='vis',model_mode='a_a_2',do_poisson=do_poisson)

        cv_lik = np.sum(np.log(lik_resp),1)

        RMSE = np.sqrt(np.mean((wrap_rad(dec_ori-ori_dec)*90/pi)**2))
        RMSE_null = np.sqrt(np.mean((wrap_rad(ori_dec-ori_current)*90/pi)**2))

        lik_null = np.sum(np.log((((resp==0)*[d_ori>0]) + ((resp==1)*[d_ori<0]))*(1-GUESS_RATE) +0.5*GUESS_RATE))

        this_dec = pd.Series({'data':data,'G':G,'params':params, 'dec_ori':dec_ori,'lik_resp':lik_resp,'dec_resp':dec_resp,
                             'neural_bias':out_neural,'percept_bias':out_percept,'do_poisson':do_poisson,'eRecon':eRecon})
        dec_subj[roi] = this_dec
    
        MF = MF.append({'subj':subj,'roi':roi,'RMSE':RMSE,'RMSE_null':RMSE_null,'lik_a_a_p':cv_lik[0],
                        'lik_a_n_p':cv_lik[1],'lik_a_a_2':cv_lik[2],'lik_null':lik_null},ignore_index=1)
    DEC_ALL[subj] = dec_subj

<ipython-input-21-5a12b18321f4>:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  DEC_ALL = pd.Series()
<ipython-input-21-5a12b18321f4>:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  dec_subj = pd.Series()


UCSD054
0.0
0.0 part 1 complete
0.0 part 2 aware complete
0.0 part 2 unaware complete
0.0 part 2 overaware complete
1.0
1.0 part 1 complete
1.0 part 2 aware complete
1.0 part 2 unaware complete
1.0 part 2 overaware complete
2.0
2.0 part 1 complete
2.0 part 2 aware complete
2.0 part 2 unaware complete
2.0 part 2 overaware complete
3.0
3.0 part 1 complete
3.0 part 2 aware complete
3.0 part 2 unaware complete
3.0 part 2 overaware complete
4.0
4.0 part 1 complete
4.0 part 2 aware complete
4.0 part 2 unaware complete
4.0 part 2 overaware complete
5.0
5.0 part 1 complete
5.0 part 2 aware complete
5.0 part 2 unaware complete
5.0 part 2 overaware complete
6.0
6.0 part 1 complete
6.0 part 2 aware complete
6.0 part 2 unaware complete
6.0 part 2 overaware complete
7.0
7.0 part 1 complete
7.0 part 2 aware complete
7.0 part 2 unaware complete
7.0 part 2 overaware complete
8.0
8.0 part 1 complete
8.0 part 2 aware complete
8.0 part 2 unaware complete
8.0 part 2 overaware complete
9.0
9.0 part 1 compl

In [22]:
DEC_ALL

UCSD054    V3    data            {'ori_prev': [[3.0479382...
UCSD060    V3    data            {'ori_prev': [[3.0479382...
UCSD061    V3    data            {'ori_prev': [[3.0479382...
UCSD062    V3    data            {'ori_prev': [[3.0479382...
UCSD063    V3    data            {'ori_prev': [[1.3356446...
UCSD064    V3    data            {'ori_prev': [[3.0479382...
dtype: object

# Save out Data

In [23]:
import pickle
this_d = {}
for subj in DEC_ALL.keys():
    this_d[subj] = dict(DEC_ALL[subj]['V3'])
# pickle.dump(this_d,open('ModelFitsV3.pkl',"wb"))

In [21]:
# Done, analyze fits in other notebook